In [1]:
import pandas as pd
import numpy as np
import datetime

data = pd.read_excel('TYLE.xlsx', sheet_name='ГРП')
dummy = pd.read_excel('TYLE.xlsx', sheet_name='МЭР')
data

,Скважина №,"Длиина ГС, м",Число стадий ГРП,"Полудлина трещины, м","Ширина трещины, мм","Проницаемость проппанта, Д",Дата ВНР после ГС \\ ГРП \\ЗБГС,ГС/ННС,Тип ГТМ,Пласт,Скважина № (скорректированная)
0,1,0.000000,0,0,0.000000,0,2016-03-01,ННС,ВНС,Ю3,1
1,1,0.000000,1,111,4.100000,100,2016-08-21,ННС+ГРП,ГРП,Ю3,1_ГРП
2,3,0.000000,0,0,0.000000,0,2011-03-01,ННС,ВНС,Ю3,3
3,3,0.000000,1,100,4.000000,100,2011-09-06,ННС+ГРП,ГРП,Ю3,3_ГРП
4,4,0.000000,0,0,0.000000,0,2006-10-01,ННС,ВНС,Ю3,4
5,4,0.000000,1,71,12.000000,100,2007-08-24,ННС+ГРП,ГРП,Ю3,4_ГРП
6,5,346.101141,0,0,0.000000,0,2018-05-01,ГС,ВНС,Ю3,5
7,6,0.000000,0,0,0.000000,0,2010-12-01,ННС,ВНС,Ю3,6
8,6,0.000000,1,100,4.000000,100,2011-08-06,ННС+ГРП,ГРП,Ю3,6_ГРП
9,7,0.000000,0,0,0.000000,0,2007-04-01,ННС,ВНС,Ю3,7


In [2]:
date_column_name = r'Дата ВНР после ГС \\ ГРП \\ЗБГС'
target = 'Скважина №'

In [3]:
#составляю список скважин с историей большей или равной 6 месяцев
dummy1 = dummy.groupby(target).agg(
    {'Дата': lambda x: 
     x.tolist()}
)
dummy1['История'] = dummy1.apply(
    lambda x: 
    1 if len(x['Дата'])>=6
    else 0, axis=1)
dummy1 = dummy1.drop(dummy1[dummy1['История'] != 1].index)
dummy1.reset_index(inplace=True)
a=dummy1['Скважина №'].tolist()
a

[1, 3, 4, 5, 6, 8]

In [4]:
#удаляю те, которых нет в полученном списке
dummy=dummy[dummy['Скважина №'].isin(a)]
dummy

,Дата,Скважина №,"Дебит жидкости, м3/сут","Дебит нефти, т/сут","Обводненность, %","Пластовое давление (ТР), атм","Забойное давление (ТР), атм",Объекты работы
0,2016-07-01,1,78.60,55.39,22.13,263.0,131.1,Ю3
1,2016-08-01,1,84.36,59.86,21.52,263.0,118.3,Ю3
2,2016-09-01,1,85.34,68.98,9.13,263.0,107.1,Ю3
3,2016-10-01,1,88.41,71.09,9.67,263.0,94.8,Ю3
4,2016-11-01,1,79.03,62.62,11.16,263.0,79.4,Ю3
...,...,...,...,...,...,...,...,...
271,2019-04-01,8,123.00,12313.00,123123.00,0.0,3123.0,Ю3
272,2019-05-01,8,13.00,123.00,123123.00,123.0,12312.0,Ю3
273,2020-01-01,8,123123.00,13.00,123123.00,123.0,123123.0,Ю3
274,2021-01-01,8,2131.00,0.00,12331.00,123.0,1231.0,Ю3


In [5]:
#провекрка скважин на которых нет грп
no_grp= data.groupby('Скважина №').agg(
    {'ГС/ННС': lambda x: 
     x.tolist()}
)
no_grp.reset_index(inplace=True)
no_grp['ГРП?'] = no_grp.apply(
    lambda x: 
    0 if 'ГРП' not in str(x['ГС/ННС'])
    else 1, axis=1)
no_grp = no_grp.drop(no_grp[no_grp['ГРП?'] == 1].index)
spisok = no_grp['Скважина №'].tolist()
spisok
#" ".join(my_list)

[5, 14, 15]

In [6]:

step = data.groupby(target).agg(
    {date_column_name: lambda x: 
     x.tolist()[0] 
     if len(x.tolist()) == 1 else x.tolist()[1]}
)
step.reset_index(inplace=True)
step=step[~step['Скважина №'].isin(spisok)].reset_index(drop=True)
step

,Скважина №,Дата ВНР после ГС \\ ГРП \\ЗБГС
0,1,2016-08-21
1,3,2011-09-06
2,4,2007-08-24
3,6,2011-08-06
4,7,2016-01-17
5,8,2015-03-13
6,9,2008-02-09
7,10,2013-02-16
8,11,2009-09-17
9,12,2008-06-26


In [7]:
dummy = dummy.merge(step, on=target)

In [8]:
dummy.head(10)

,Дата,Скважина №,"Дебит жидкости, м3/сут","Дебит нефти, т/сут","Обводненность, %","Пластовое давление (ТР), атм","Забойное давление (ТР), атм",Объекты работы,Дата ВНР после ГС \\ ГРП \\ЗБГС
0,2016-07-01,1,78.60,55.39,22.13,263.0,131.1,Ю3,2016-08-21
1,2016-08-01,1,84.36,59.86,21.52,263.0,118.3,Ю3,2016-08-21
2,2016-09-01,1,85.34,68.98,9.13,263.0,107.1,Ю3,2016-08-21
3,2016-10-01,1,88.41,71.09,9.67,263.0,94.8,Ю3,2016-08-21
4,2016-11-01,1,79.03,62.62,11.16,263.0,79.4,Ю3,2016-08-21
5,2016-12-01,1,68.39,53.63,12.20,263.0,69.8,Ю3,2016-08-21
6,2017-01-01,1,57.06,42.88,16.31,236.0,83.5,Ю3,2016-08-21
7,2011-07-01,3,150.48,125.65,5.71,203.0,120.5,Ю3,2011-09-06
8,2011-08-01,3,151.53,126.52,5.72,203.0,104.2,Ю3,2011-09-06
9,2011-09-01,3,157.68,133.03,4.59,203.0,103.0,Ю3,2011-09-06


In [9]:
dummy[target] = dummy.apply(
    lambda x: 
    str(x[target])
    + ('_ГРП' if x['Дата'] > x[date_column_name] else ''),
    axis=1
)
dummy = dummy.drop(date_column_name, axis=1)

In [10]:
dummy.head(10)

,Дата,Скважина №,"Дебит жидкости, м3/сут","Дебит нефти, т/сут","Обводненность, %","Пластовое давление (ТР), атм","Забойное давление (ТР), атм",Объекты работы
0,2016-07-01,1,78.60,55.39,22.13,263.0,131.1,Ю3
1,2016-08-01,1,84.36,59.86,21.52,263.0,118.3,Ю3
2,2016-09-01,1_ГРП,85.34,68.98,9.13,263.0,107.1,Ю3
3,2016-10-01,1_ГРП,88.41,71.09,9.67,263.0,94.8,Ю3
4,2016-11-01,1_ГРП,79.03,62.62,11.16,263.0,79.4,Ю3
5,2016-12-01,1_ГРП,68.39,53.63,12.20,263.0,69.8,Ю3
6,2017-01-01,1_ГРП,57.06,42.88,16.31,236.0,83.5,Ю3
7,2011-07-01,3,150.48,125.65,5.71,203.0,120.5,Ю3
8,2011-08-01,3,151.53,126.52,5.72,203.0,104.2,Ю3
9,2011-09-01,3,157.68,133.03,4.59,203.0,103.0,Ю3


In [11]:
#удаляем стоки с нулевым дебитом нефти
dummy = dummy.drop(dummy[dummy['Дебит нефти, т/сут'] == 0].index)
dummy

,Дата,Скважина №,"Дебит жидкости, м3/сут","Дебит нефти, т/сут","Обводненность, %","Пластовое давление (ТР), атм","Забойное давление (ТР), атм",Объекты работы
0,2016-07-01,1,78.60,55.39,22.13,263.0,131.1,Ю3
1,2016-08-01,1,84.36,59.86,21.52,263.0,118.3,Ю3
2,2016-09-01,1_ГРП,85.34,68.98,9.13,263.0,107.1,Ю3
3,2016-10-01,1_ГРП,88.41,71.09,9.67,263.0,94.8,Ю3
4,2016-11-01,1_ГРП,79.03,62.62,11.16,263.0,79.4,Ю3
...,...,...,...,...,...,...,...,...
242,2019-03-01,8_ГРП,331.00,3123.00,123.00,123.0,123.0,Ю3
243,2019-04-01,8_ГРП,123.00,12313.00,123123.00,0.0,3123.0,Ю3
244,2019-05-01,8_ГРП,13.00,123.00,123123.00,123.0,12312.0,Ю3
245,2020-01-01,8_ГРП,123123.00,13.00,123123.00,123.0,123123.0,Ю3


In [12]:
#если нулевое пластовое давление, то беру предыдущее значение
dummy['Предыдущее пластовое давление']=dummy['Пластовое давление (ТР), атм'].shift(-1)

dummy['Пластовое давление (ТР), атм'],dummy['Предыдущее пластовое давление']=np.where(dummy['Пластовое давление (ТР), атм']==0,(dummy['Предыдущее пластовое давление'],dummy['Пластовое давление (ТР), атм']),(dummy['Пластовое давление (ТР), атм'],dummy['Предыдущее пластовое давление']))
dummy = dummy.drop('Предыдущее пластовое давление', axis=1)
dummy        

,Дата,Скважина №,"Дебит жидкости, м3/сут","Дебит нефти, т/сут","Обводненность, %","Пластовое давление (ТР), атм","Забойное давление (ТР), атм",Объекты работы
0,2016-07-01,1,78.60,55.39,22.13,263.0,131.1,Ю3
1,2016-08-01,1,84.36,59.86,21.52,263.0,118.3,Ю3
2,2016-09-01,1_ГРП,85.34,68.98,9.13,263.0,107.1,Ю3
3,2016-10-01,1_ГРП,88.41,71.09,9.67,263.0,94.8,Ю3
4,2016-11-01,1_ГРП,79.03,62.62,11.16,263.0,79.4,Ю3
...,...,...,...,...,...,...,...,...
242,2019-03-01,8_ГРП,331.00,3123.00,123.00,123.0,123.0,Ю3
243,2019-04-01,8_ГРП,123.00,12313.00,123123.00,123.0,3123.0,Ю3
244,2019-05-01,8_ГРП,13.00,123.00,123123.00,123.0,12312.0,Ю3
245,2020-01-01,8_ГРП,123123.00,13.00,123123.00,123.0,123123.0,Ю3


In [13]:
dummy.to_excel('tyle3333.xlsx')